In [1]:
import azureml.core
from azureml.core import Workspace
ws = Workspace.from_config()
print(ws.name)

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (cryptography 3.1.1 (c:\users\tonyl\anaconda3\lib\site-packages), Requirement.parse('cryptography<4.0.0,>=3.3.1; extra == "crypto"'), {'PyJWT'}).


dp-100-01


In [26]:
from azureml.core import Environment
myenv = Environment("user-managed-env")
myenv.python.user_managed_dependencies = True

In [2]:
import os, shutil
training_folder = 'data/diabetes-training'
os.makedirs(training_folder, exist_ok=True)

shutil.copy('data/data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

'data/diabetes-training\\diabetes.csv'

In [6]:
%%writefile training_folder/diabetes_training.py
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [7]:
run = Run.get_context()

In [8]:
diabetes = pd.read_csv('data/diabetes-training/diabetes.csv')
X, y = diabetes[['Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure', 'TricepsThickness', 'SerumInsulin', 'BMI', 'DiabetesPedigree',
                'Age']].values, diabetes['Diabetic'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [9]:
reg = 0.01
run.log('Regularization Rate', np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

Attempted to log scalar metric Regularization Rate:
0.01


In [19]:
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_scores[:,1])
run.log('AUC', np.float(auc))

In [21]:
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## Use an Estimator to Run the Script as an Experiment
You can run experiment scripts using a RunConfiguration and a ScriptRunConfig, or you can use an Estimator, which abstracts both of these configurations in a single object.

In this case, we'll use a generic Estimator object to run the training experiment. Note that the default environment for this estimator does not include the scikit-learn package, so you need to explicitly add that to the configuration. The conda environment is built on-demand the first time the estimator is used, and cached for future runs that use the same configuration; so the first run will take a little longer. On subsequent runs, the cached environment can be re-used so they'll complete more quickly.

In [32]:
#from azureml.train.estimator import Estimator
from azureml.core import ScriptRunConfig
from azureml.core import Experiment

# Create an estimator
estimator = ScriptRunConfig(
                      source_directory='training_folder',
                      script='diabetes_training.py',
                      compute_target='local',
                        environment=myenv
                      )

# Create an experiment
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)


# Run the experiment based on the estimator
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

RunId: diabetes-training_1620854028_58055822
Web View: https://ml.azure.com/runs/diabetes-training_1620854028_58055822?wsid=/subscriptions/535a47ed-8a9f-40a9-a3fb-7c680cbacbc5/resourcegroups/dp100/workspaces/dp-100-01&tid=78aac226-2f03-4b4d-9037-b46d56c55210

Streaming azureml-logs/60_control_log.txt

[2021-05-12T21:13:51.858349] Entering context manager injector.
[2021-05-12T21:13:53.377665] Using urllib.request Python 3.0 or later
Streaming log file azureml-logs/60_control_log.txt
Running: ['cmd.exe', '/c', 'C:\\Users\\tonyl\\AppData\\Local\\Temp\\azureml_runs\\diabetes-training_1620854028_58055822\\azureml-environment-setup/conda_env_checker.bat']

Streaming azureml-logs/70_driver_log.txt

[2021-05-12T21:13:54.323355] Entering context manager injector.
[context_manager_injector.py] Command line Options: Namespace(inject=['ProjectPythonPath:context_managers.ProjectPythonPath', 'RunHistory:context_managers.RunHistory', 'TrackUserError:context_managers.TrackUserError'], invocation=['di

{'runId': 'diabetes-training_1620854028_58055822',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-05-12T21:13:53.604066Z',
 'endTimeUtc': '2021-05-12T21:14:29.579999Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '414cef39-3291-4982-81f8-fed6e4db2e8f'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'diabetes_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'user-managed-env',
   'version': 'Autosave_2021-05-12T21:02:49Z_571891c8',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': True,
    'condaDependencies': {'channels

In [33]:
from azureml.widgets import RunDetails

RunDetails(run).show()

# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Regularization Rate 0.01
Accuracy 0.774
AUC 0.8484377332205582


azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/8884_azureml.log
outputs/diabetes_model.pkl


In [34]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Estimator'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabetes_model version: 1
	 Training context : Estimator
	 AUC : 0.8484377332205582
	 Accuracy : 0.774




In [35]:
run.get_metrics()

{'Regularization Rate': 0.01, 'Accuracy': 0.774, 'AUC': 0.8484377332205582}

## Create a Parameterized Training Script + Overwriting script file
You can increase the flexibility of your training experiment by adding parameters to your script, enabling you to repeat the same training experiment with different settings. In this case, you'll add a parameter for the regularization rate used by the Logistic Regression algorithm when training the model.

Again, lets start by creating a folder for the parameterized script and the training data.

In [36]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'diabetes-training-params'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

'diabetes-training-params\\diabetes.csv'

In [37]:
%%writefile $training_folder/diabetes_training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Get the experiment run context
run = Run.get_context()

# Set regularization hyperparameter
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
args = parser.parse_args()
reg = args.reg

# load the diabetes dataset
print("Loading Data...")
# load the diabetes dataset
diabetes = pd.read_csv('diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

Writing diabetes-training-params/diabetes_training.py


## Use a Framework-Specific Estimator
You used a generic Estimator class to run the training script, but you can also take advantage of framework-specific estimators that include environment definitions for common machine learning frameworks. In this case, you're using Scikit-Learn, so you can use the SKLearn estimator. This means that you don't need to specify the scikit-learn package in the configuration.

Note: Once again, the training experiment uses a new environment; which must be created the first time it is run.

In [43]:

#from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails

# Create an estimator
estimator = ScriptRunConfig(source_directory='training_folder',
                    script='diabetes_training.py',
                    arguments = ['--reg_rate', 0.1],
                    compute_target='local',
                    environment=myenv
                    )

# Create an experiment
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment
run = experiment.submit(config=estimator)

# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()



_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'diabetes-training_1620854711_12219912',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2021-05-12T21:25:15.959157Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '22758121-0a8b-4106-ae98-ddf8b5f5feb1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'diabetes_training.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--reg_rate', '0.1'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'user-managed-env',
   'version': 'Autosave_2021-05-12T21:02:49Z_571891c8',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': True,
    'condaDependencies': {'channels': ['anaconda', 'conda-for

In [44]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Regularization Rate 0.1
Accuracy 0.774
AUC 0.8484427085924674


azureml-logs/60_control_log.txt
azureml-logs/70_driver_log.txt
logs/azureml/16548_azureml.log
outputs/diabetes_model.pkl



## Register A New Version of the Model
Now that you've trained a new model, you can register it as a new version in the workspace.

In [ ]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Parameterized SKLearn Estimator'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')